# SRT Parser

srt is a format of txt files which are used as subtitles for movies.
Looking at vast resources of subtitles in various languages, I thought it could be valuable source for NLP models especially those requiring translation. Source for that is will be webpage 
https://www.opensubtitles.com/en/tvshows/2009-lie-to-me/seasons/2/episodes/1-lie-to-me-the-core-of-it
and the tv serie is " Lie to me"

To manipualate those files globally there is really good library
which could be installed 

```
 !pip install -U git+https://github.com/cdown/srt.git@develop
```
At the end of the day result shoul dlook like a csv file or pandas wher in one column will be polish sentence and other will be english one.



operations were done on local conda environment

## Load libraries

In [ ]:
import csv
import os
import string
import pandas as pd

## 1. Converting srt to csv

In [ ]:
filePath = "./"

dirList = os.listdir(filePath)
extension = str(os.path.splitext(filename)[1])
root = str(os.path.splitext(filename)[0])

for filename in dirList:
	timestamps = ["timestamp"] #the complete timestamp of each subtitle
	timeStarts = ["start time"] #when each subtitle appears on screen
	timeEnds = ["end time"] #when each subtitle is removed from the screen
	subtitleIndices = ["index"]
	utterances = ["utterance"]
	extension = str(os.path.splitext(filename)[1])
	root = str(os.path.splitext(filename)[0])
	if extension != ".srt": #skip files other than subtitle files
		continue
	currentFile = open(filename)
	prevLineWasUtt = False #was the most recent line looked at an utterance?
	for line in currentFile: #populate the lists that will be put in the csv
		line = line.strip() #get rid of whitespace characters on end of line
		if line.find("-") == 0: #prevents Excel from viewing entry as formula
			line = "\'" + line
		if line.isdigit():
			subtitleIndices.append(line)
		elif "-->" in line:
			timestamps.append(line)
			timeStarts.append(line[:8])
			timeEnds.append(line[17:25])
		elif len(line) > 1:
			if prevLineWasUtt:
				utterances[len(utterances)-1] = utterances[len(utterances)-1] + " " + line
			else:
				utterances.append(line)
			prevLineWasUtt = True
			continue
		prevLineWasUtt = False		
	#put all the lists in a csv file with the same name as the subtitle file
	csvWriter = csv.writer(open('./input/'+root + ".csv",'w'))
	for nr in range(len(subtitleIndices)):
		csvWriter.writerow([subtitleIndices[nr], timestamps[nr], timeStarts[nr], timeEnds[nr], utterances[nr]])

In [ ]:
df_pl= pd.read_csv('./input/Lie-pl-S01E01.csv')
df_en= pd.read_csv('./input/Lie-en-S01E01.csv')

In [ ]:
df_pl.head(6)

,index,timestamp,start time,end time,utterance
0,1,"00:00:01,000 --> 00:00:30,000",00:00:01,00:00:30,www.napiprojekt.pl - nowa jakość napisów. Napi...
1,2,"00:02:51,000 --> 00:03:25,000",00:02:51,00:03:25,"Poinstruowałem mojego klienta, aby nie odpowia..."
2,3,"00:05:00,000 --> 00:05:18,000",00:05:00,00:05:18,Nie będzie mówił.
3,4,"00:05:20,000 --> 00:05:33,000",00:05:20,00:05:33,W porządku.
4,5,"00:05:34,000 --> 00:06:04,000",00:05:34,00:06:04,Sam nie mam wielkiej wiary w słowa.
5,6,"00:06:14,000 --> 00:07:09,000",00:06:14,00:07:09,Przeciętna osoba kłamie średnio trzy razy w ci...


In [ ]:
df_en.head(6)

,index,timestamp,start time,end time,utterance
0,1,"00:00:15,054 --> 00:00:18,350",00:00:15,00:00:18,I've instructed my client to remain silent.
1,2,"00:00:28,047 --> 00:00:29,633",00:00:28,00:00:29,He's not gonna talk.
2,3,"00:00:30,907 --> 00:00:32,065",00:00:30,00:00:32,That's okay.
3,4,"00:00:32,304 --> 00:00:35,040",00:00:32,00:00:35,I don't have much faith in words myself.
4,5,"00:00:36,459 --> 00:00:39,348",00:00:36,00:00:39,"Now, statistically speaking, the average perso..."
5,6,"00:00:39,555 --> 00:00:42,413",00:00:39,00:00:42,three lies per 10 minutes of conversation.


## 2. Manipulating content of pd 

### Functions

In [ ]:
def kill_col(df):
        if 'level_0' in df:
            del df['level_0']
#         print()
#         if 'index' in df:
#             df= df.drop(['index'], axis=1)

            
        return df

In [ ]:
def kill_row(df, row_nr):

    df=kill_col(df)
    print('row nr ',row_nr, df.utterance[row_nr], '---------->will be erased' )
    df=df.drop(df.index[[row_nr]])
    df=df.reset_index()
#     df= df.drop(['level_0'], axis=1)
    return df   

In [ ]:
def paralel_display(df1, df2,row_st, row_end):
    for i in range(row_st, row_end, 1):
        print((df1.utterance[i]).rjust(75),'|',str(i).rjust(3),'|', df2.utterance[i])

In [ ]:
def concat_two_rows(df, row_nr):
    
    df.loc[row_nr,'utterance']=df.utterance[row_nr]+df.utterance[row_nr+1]
    df= kill_row(df, row_nr+1)
    
    return df

In [ ]:
# Wrting string to particular location
# def write_to(df, row_nr):
#     df.loc['row_nr','utterance']='something '
#     return df

### Ajusting content

In [ ]:
df_pl= kill_row(df_pl, 266)

row nr  266 Kongresman Zeb Weil którego karierę budowałem przez 20 lat, ---------->will be erased


In [ ]:
df_en= kill_row(df_en,266)

row nr  266 '- He's a better liar than all of us. '- I need your help, Gillian.Congressman Zeb Weil, whose career I spent 20 years building, ---------->will be erased


In [ ]:
# =============================================

In [ ]:
paralel_display(df_en, df_pl,290, 300)

                                       '- James took these? '- Yes, he did. | 290 | '- Zrobił je James? '- Tak.
       We let him convert that bathroom to a dark room for his photography. | 291 | Pozwoliliśmy zmienić łazienkę w ciemnię.
                                                         Is that a problem? | 292 | To był jakiś problem.
It was taking too much time from his responsibilitiesas a Witness doing God's work. | 293 | Zabierało mu ta za dużo czasu. Powinien go spędzać jako świadek bożego dzieła.
                              What did James tell you about Miss McCartney? | 294 | Co James mówił o pani McCartney?
                                         He thought she was a good teacher. | 295 | Mówił, że była dobrą nauczycielką.
                                                               And did you? | 296 | Co Pan o tym sądzi?
                                             James did not kill that woman. | 297 | James nie zabił tej kobiety.
                He would not take

In [ ]:
df_pl=concat_two_rows(df_pl,288)

row nr  289 Nie rozumiem w jakim celu chcecie obejrzeć jego sypialnię. ---------->will be erased


In [ ]:
df_en=concat_two_rows(df_en, 293)

row nr  294 as a Witness doing God's work. ---------->will be erased


In [ ]:
# wvvzytaj h kolo 8 jest problem i zacznij od 300

## 3. Saving results

In [ ]:
df_en.to_hdf("2_en.h5", 'df_en')
df_pl.to_hdf("2_pl.h5", 'df_pl')

In [ ]:
df_en=pd.read_hdf("2_en.h5", 'df_en')
df_pl=pd.read_hdf("2_pl.h5", 'df_pl')

Renaming columns to avoid problems after concataenation

In [ ]:
df_en.rename(columns={'utterance':'en'}, inplace=True)
df_pl.rename(columns={'utterance':'pl'}, inplace=True)

In [ ]:
df_en.head(3)

,level_0,index,timestamp,start time,end time,en
0,0,1,"00:00:36,900 --> 00:00:39,230",00:00:36,00:00:39,Cal lightman sees the truth.
1,1,2,"00:00:39,296 --> 00:00:41,492",00:00:39,00:00:41,It's written all over your face.
2,2,3,"00:00:41,559 --> 00:00:43,920",00:00:41,00:00:43,"It's also in your voice, your posture,"


In [ ]:
df_pl.head(3)

,level_0,index,timestamp,start time,end time,pl
0,0,3,"00:00:36,995 --> 00:00:39,039",00:00:36,00:00:39,Cal Lightman widzi prawdę.
1,1,4,"00:00:39,998 --> 00:00:41,875",00:00:39,00:00:41,Jest wypisana na waszych twarzach.
2,2,5,"00:00:41,958 --> 00:00:44,878",00:00:41,00:00:44,"Jest w waszym głosie, w waszych ruchach,"


In [ ]:
# but yuo need rename columns utterance

In [ ]:
df3=pd.concat([df_en, df_pl], axis=1)

In [ ]:
keep_list=['en','pl']

In [ ]:
for l in list(df3.columns):
    if l not in keep_list:
        del df3[l]

In [ ]:
df3.head(10)

,en,pl
0,Cal lightman sees the truth.,Cal Lightman widzi prawdę.
1,It's written all over your face.,Jest wypisana na waszych twarzach.
2,"It's also in your voice, your posture,","Jest w waszym głosie, w waszych ruchach,"
3,The words you choose.,"W słowach, które dobieracie."
4,"Because of dr. lightman's scientific discoveries,","Z powodu naukowych odkryć, dra Lightmana"
5,We know that these signs are the same...,"Mamy świadomość, że te znaki są takie same dla..."
6,"Laying it on a bit thick, isn't she?","Nieźle podmaśla, co nie?"
7,"It's called selling books, dad. shut up and bask.","To się nazywa promocja, tato. Ucisz się i rozk..."
8,"Now, give him 5 minutes and 20 questions,","A teraz, macie 5 minut i 20 pytań,"
9,And he'll know whether you went off to argenti...,"A on będzie wiedział, czy byliście w Argentyni..."


In [ ]:
df_en.to_hdf("./output/1_enpl.h5", 'df3') 